### Alignment w/ more aligning data

In [232]:
from silnlp.common.corpus import load_corpus, write_corpus
from silnlp.alignment.config import get_aligner
from silnlp.alignment.machine_aligner import MachineAligner
from typing import List, Union
import tempfile
from pathlib import Path
from machine.tokenization import LatinWordTokenizer
from machine.corpora import escape_spaces, nfc_normalize, lowercase

# Tokenize and normalize sentences in the same way as the normal alignment process
def tokenize_for_alignment(sents: List[Union[str, List[str]]]) -> List[str]:
    if type(sents[0]) == str:
        tokenizer = LatinWordTokenizer()
        sents = [tokenizer.tokenize(sent) for sent in sents]
    sents_norm = [lowercase(nfc_sent) for nfc_sent in [nfc_normalize(es_sent) for es_sent in [escape_spaces(sent) for sent in sents]]]
    return [" ".join(toks) for toks in sents_norm]

def align_sents(src_sents: List[str],
                trg_sents: List[str],
                aligner_id: str = "hmm",
                sym_align_path: Path = None,
                extra_train_data_src: Path = None,
                extra_train_data_trg: Path = None
                ) -> None:
    with tempfile.TemporaryDirectory() as td:
        temp_dir = Path(td)

        # Since the sentences get tokenized before they are normalized in the normal alignment process,
        # we can wait to do the normalization until here
        src_sents_norm = tokenize_for_alignment(src_sents)
        trg_sents_norm = tokenize_for_alignment(trg_sents)

        # Prep alignment data
        align_src_path = temp_dir / "align.src.txt"
        align_trg_path = temp_dir / "align.trg.txt"
        write_corpus(align_src_path, src_sents_norm)
        write_corpus(align_trg_path, trg_sents_norm)

        # Prep training data
        if extra_train_data_src and extra_train_data_trg:
            src_sents_norm += tokenize_for_alignment(list(load_corpus(extra_train_data_src)))
            trg_sents_norm += tokenize_for_alignment(list(load_corpus(extra_train_data_trg)))
        train_src_path = temp_dir / "train.src.txt"
        train_trg_path = temp_dir / "train.trg.txt"
        write_corpus(train_src_path, src_sents_norm)
        write_corpus(train_trg_path, trg_sents_norm)

        # Train the aligner and align
        aligner: MachineAligner = get_aligner(aligner_id, temp_dir)

        aligner.train(train_src_path, train_trg_path)
        aligner.force_align(align_src_path, align_trg_path, sym_align_path)

### USFM marker preservation
* Extract footnotes and put them at the end
* Extract each instance of a marker and record its index
* Tokenize source sentences and match each marker to surrounding tokens based on their original indices
* Train aligner on all training data + translation, align translation to source
* Reinsert marker instances

In [233]:
from pathlib import Path

'''Define project values'''
pair = ""
project = ""
file_suffix = ""
trg_project = ""
trg_file_suffix = ""

book = "MAT"
book_name = f"41{book}"
draft = True

src_fpath = Path(f"test_S/Paratext/projects/{project}/{book_name}{file_suffix}.SFM")
if draft:
    trg_fpath = Path(f"zzz_USFM/{pair}/{book}/{book_name}_draft.SFM")
else:
    trg_fpath = Path(f"test_S/Paratext/projects/{trg_project}/{book_name}{trg_file_suffix}.SFM")
aligner = "eflomal"
pair_book_dir = Path(f"zzz_USFM/{pair}/{book}")
align_path = pair_book_dir / f"{book_name}_sym-align_{aligner}.txt"
out_fpath = pair_book_dir / f"{book_name}{trg_file_suffix}_out.SFM"

In [ ]:
from machine.corpora import FileParatextProjectSettingsParser, UsfmFileText, UsfmTokenizer
from machine.tokenization import LatinWordTokenizer

src_settings = FileParatextProjectSettingsParser(src_fpath.parent).parse()
src_file_text = UsfmFileText(
    src_settings.stylesheet,
    src_settings.encoding,
    book,
    src_fpath,
    src_settings.versification,
    include_markers=True, # F/T gives notes their own rows, F/F gives just the main text, T/F gives one ref per verse and all markers are inline
    include_all_text=True, # T/T includes all intro and section titles (as does F/T), all other notes/markers inline
    project=src_settings.name,
)

sentences = []
vrefs = []
for sent in src_file_text:
    if (len(sent.ref.path) == 0 or sent.ref.path[-1].name != "rem") and len(sent.text.strip()) > 0:
        sentences.append(sent.text.strip())
        vrefs.append(sent.ref)
        print(vrefs[-1], sentences[-1])

# for ref, sent in zip(vrefs, sentences):
#     print(ref, sent)

### Only deal with paragraph markers

In [ ]:
from machine.corpora import UsfmTokenizer, UsfmTokenType, UsfmStyleType
# TODO: would it be easier to always use StyleType (for UsfmTags) vs TokenType (for UsfmTokens)?

'''Parse sentences'''
tokenizer = UsfmTokenizer(src_settings.stylesheet)
sentence_toks = [tokenizer.tokenize(sent) for sent in sentences]

to_delete = ["fig"]
inline_markers = []
# markers_by_verse = [[] for _ in sentence_toks]
text_only_sents = ["" for _ in sentence_toks]
ignored_segments = []
for i, (toks, ref) in enumerate(zip(sentence_toks, vrefs)):
    ignored_segment = ""
    ignore_scope = None
    for j, tok in enumerate(toks): # POSSIBLE TYPES: TEXT, PARAGRAPH, CHARACTER, NOTE, END
        if ignore_scope is not None:
            ignored_segment += tok.to_usfm()
            if tok.type == UsfmTokenType.END and tok.marker[:-1] == ignore_scope.marker:
                ignored_segments.append((ref, ignored_segment))
                ignored_segment = ""
                ignore_scope = None
        elif tok.type == UsfmTokenType.NOTE or (tok.type == UsfmTokenType.CHARACTER and tok.marker in to_delete):
            ignore_scope = tok
            ignored_segment += tok.to_usfm()
        elif tok.type in [UsfmTokenType.PARAGRAPH, UsfmTokenType.CHARACTER, UsfmTokenType.END]: # possible tok.marker forms: w w* +w +w*
            # len of text so far == start idx of next text tok, this won't break with other markers bc the idx is wrt only the text toks 
            inline_markers.append((i, len(text_only_sents[i]), tok.to_usfm())) # previously tok.to_usfm() so it could be inserted into the text
        elif tok.type == UsfmTokenType.TEXT:
            text_only_sents[i] += tok.text
        # elif tok.type != UsfmTokenType.CHARACTER and tok.type != UsfmTokenType.END:
        #     print(tok.type, tok)

# print("sent_idx, orig_idx, marker")
# for marker in inline_markers:
#     print(marker)
# for ref, sent in zip(vrefs, text_only_sents):
#     print(ref, sent)

In [ ]:
from machine.corpora import FileParatextProjectSettingsParser, UsfmFileText
from pathlib import Path

'''Translate sentences (or use target sentences for testing)'''

# Get target file and remove all markers
if draft:
    trg_file_text = UsfmFileText(
        src_settings.stylesheet,
        src_settings.encoding,
        book,
        trg_fpath,
        src_settings.versification,
        include_markers=True,
        include_all_text=True,
        project=src_settings.name,
    )
else:
    trg_settings = FileParatextProjectSettingsParser(trg_fpath.parent).parse()
    trg_file_text = UsfmFileText(
        trg_settings.stylesheet,
        trg_settings.encoding,
        trg_settings.get_book_id(trg_fpath.name),
        trg_fpath,
        trg_settings.versification,
        include_markers=True,
        include_all_text=True,
        project=trg_settings.name,
    )
    tokenizer = UsfmTokenizer(trg_settings.stylesheet)

trg_sents = []
trg_vrefs = []
for sent in trg_file_text:
    if (len(sent.ref.path) > 0 and sent.ref.path[-1].name == "rem") or len(sent.text.strip()) == 0:
        continue
    trg_sents.append("")
    trg_vrefs.append(sent.ref)

    sent = sent.text.strip()
    usfm_toks = tokenizer.tokenize(sent)
    ignore_scope = None
    for j, tok in enumerate(usfm_toks): # POSSIBLE TYPES: TEXT, PARAGRAPH, CHARACTER, NOTE, END
        if ignore_scope is not None:
            if tok.type == UsfmTokenType.END and tok.marker[:-1] == ignore_scope.marker:
                ignore_scope = None
        elif tok.type == UsfmTokenType.NOTE or (tok.type == UsfmTokenType.CHARACTER and tok.marker in to_delete):
            ignore_scope = tok
        elif tok.type == UsfmTokenType.TEXT:
            trg_sents[-1] += tok.text

# for ref, sent in zip(trg_vrefs, trg_sents):
#     print(ref, sent)

In [237]:
from typing import Tuple, List

'''Tokenize sentences'''
tokenizer = LatinWordTokenizer()

# TODO: force ranges to not cross boundaries by tokenizing each text token separately? or bound the alignment range some other way based on surrounding markers
# is there a simpler way to force markers to be reinserted in the same relative order?
word_tok_ranges = [list(tokenizer.tokenize_as_ranges(sent)) for sent in text_only_sents]
toks = [[sent[r.start : r.end] for r in ranges] for sent, ranges in zip(text_only_sents, word_tok_ranges)]

trg_word_tok_ranges = [list(tokenizer.tokenize_as_ranges(sent)) for sent in trg_sents]
trg_toks = [[sent[r.start : r.end] for r in ranges] for sent, ranges in zip(trg_sents, trg_word_tok_ranges)]

'''Match markers to the token closest to them'''
# TODO: need to disambiguate the order of markers in the case where there are multiple markers in a row
# Returns a list of the indices of the the tokens following each of the input sequences
# The returned indices are based on the tokens post-marker removal, but the lookup is based on the character indices of the original strings
def get_toks_after_sequences(sequences: List[Tuple]) -> List[int]:
    toks_after_seqs = []
    for sequence in sequences:
        sent_idx, start_idx = sequence[0], sequence[1] # now, start_idx is wrt text_only_sents
        for i, tok_range in reversed(list(enumerate(word_tok_ranges[sent_idx]))):
            # this works fine but there's still risk of splitting words if the token goes across paragraph boundaries
            # this can be fixed by forcing tok boundaries to stay within usfm text toks
            if tok_range.start < start_idx:
                toks_after_seqs.append(i+1)
                break
            if i == 0:
                toks_after_seqs.append(i)

    return toks_after_seqs
toks_after_markers = get_toks_after_sequences(inline_markers)

In [238]:
from silnlp.alignment.utils import compute_alignment_scores
from silnlp.common.corpus import write_corpus
'''Align sentences'''
# align_sents(toks, trg_toks, aligner, align_path) # Path(f"zzz_PN_KTs/tpi_aps/train.src.detok.txt"),Path(f"zzz_PN_KTs/tpi_aps/train.trg.detok.txt")

# eflomal
if not align_path.exists():
    write_corpus(pair_book_dir / "src_align.txt", text_only_sents) # orig sentences
    write_corpus(pair_book_dir / "trg_align.txt", trg_sents)
    compute_alignment_scores(pair_book_dir / "src_align.txt", pair_book_dir / "trg_align.txt", aligner, align_path)

In [ ]:
from collections import defaultdict

from silnlp.common.corpus import load_corpus

# probably non-exhaustive, second of same-looking single closing quotes is an apostrophe, I think
# QUOTATION_MARKS = ["'", '"', "“", "”", "‘", "’", "ʼ", "<", ">"]

'''Decide where to reinsert markers'''

# read aligner output
if aligner == "eflomal":
    align_lines = [[(lambda x: (int(x[0]), int(x[1])))(pair.split("-")) for pair in line.split()] for line in load_corpus(align_path)]
else:
    align_lines = [[(lambda x: (int(x[0]), int(x[1])))(pair.split(":")[0].split("-")) for pair in line.split()] for line in load_corpus(align_path)]

# Gets the number of alignment pairs that "cross the line" between (src_idx - .5) and (trg_idx - .5)
def num_align_crossings(sent_idx: int, src_idx: int, trg_idx: int) -> int:
    crossings = 0
    for i,j in align_lines[sent_idx]:
        if i < src_idx and j >= trg_idx:
            crossings += 1
        if i >= src_idx and j < trg_idx:
            crossings += 1
    return crossings

# Get the index of the trg token that each sequence should be inserted before
def get_insert_indices(seqs_to_insert: List[Tuple], adj_tok_idxs: List[int]) -> List[int]:
    punct_hyp_freqs = defaultdict(lambda: defaultdict(int))
    hyp_freqs = defaultdict(lambda: defaultdict(int))

    insert_indices = []
    for seq, adj_src_tok in zip(seqs_to_insert, adj_tok_idxs):
        sent_idx = seq[0]
        marker = seq[2].strip(" \\+")

        '''Original, checks first hypothesis of its original position and one tok previous and picks the better of the two'''
        # If the token on either side of a hypothesis is punctuation, use that
        # can also try a less extreme version where the punct hyps are still subject to having the least crossings, but they are still always checked first

        trg_hyp = -1
        punct_hyps = [-1, 0]
        for punct_hyp in punct_hyps:
            src_hyp = adj_src_tok + punct_hyp
            if src_hyp < 0 or src_hyp >= len(toks[sent_idx]):
                continue
            # only accept pairs where both the src and trg token are punct
            # can define more specifically what the punct tokens can look like later
            if len(toks[sent_idx][src_hyp]) > 0 and not any(c.isalpha() for c in toks[sent_idx][src_hyp]):
                align_pairs = reversed(align_lines[sent_idx]) if punct_hyp < 0 else align_lines[sent_idx]
                for s,t in align_pairs:
                    if s == src_hyp and not any(c.isalpha() for c in trg_toks[sent_idx][t]):
                        trg_hyp = t
                        break
            if trg_hyp != -1:
                # if this search gets expanded beyond [-1,0] can do insert_idx -= punct_hyp
                insert_idx = trg_hyp + 1 if punct_hyp < 0 else trg_hyp
                insert_indices.append(insert_idx)
                punct_hyp_freqs[marker][punct_hyp] += 1
                break
        if trg_hyp != -1:
            continue

        hyps = [0, 1, 2]
        best_hyp = (-1, None, len(align_lines[sent_idx]))
        checked = set() # to prevent checking the same idx twice
        for hyp in hyps:
            align_pairs = reversed(align_lines[sent_idx]) if hyp < 0 else align_lines[sent_idx]
            src_hyp = adj_src_tok + hyp
            if src_hyp in checked:
                continue
            trg_hyp = -1
            while trg_hyp == -1 and src_hyp >= 0 and src_hyp < len(toks[sent_idx]):
                checked.add(src_hyp)
                trg_hyps = [t for (s,t) in align_pairs if s == src_hyp]
                if len(trg_hyps) > 0:
                    trg_hyp = trg_hyps[0]
                else:
                    src_hyp += -1 if hyp < 0 else 1
            if trg_hyp != -1:
                num_crossings = num_align_crossings(sent_idx, src_hyp, trg_hyp)
                if num_crossings < best_hyp[2]:
                    # replace hyp with src_hyp - adj_src_tok
                    # above is what I was trying to do before with offsetting the farther away hyps, but it's worse
                    # am I doing this below now with "insert_idx = insert_idx - best_hyp[1]"?
                    best_hyp = (trg_hyp, hyp, num_crossings)

        if best_hyp[0] == -1:
            insert_indices.append(len(trg_toks[sent_idx])) # insert at the end of the sentence
            hyp_freqs[marker]["none"] += 1
        else:
            # insert_idx = best_hyp[0]
            # # do this before or after subtracting the offset? only subtract offset if not adjacent to punct? check before and after subtraction?
            # if trg_toks[sent_idx][insert_idx] in [",", ".", "!", "?"]:
            #     insert_idx += 1
            # # subtracting hyp at the end may be bad in other cases, or make wrong answers worse/more confusing
            # else:

            # insert_idx = insert_idx - best_hyp[1]
            # if trg_toks[sent_idx][insert_idx] in [",", ".", "!", "?"]:
            #     insert_idx += 1

            insert_indices.append(best_hyp[0])
            # for end markers, preventing double adding (from punct first) was almost the same but slightly worse
            hyp_freqs[marker][best_hyp[1]] += 1

    print({k: dict(v) for k,v in punct_hyp_freqs.items()})
    print({k: dict(v) for k,v in hyp_freqs.items()})
    
    return insert_indices

trg_toks_after_markers = get_insert_indices(inline_markers, toks_after_markers)

print(toks_after_markers)
print(trg_toks_after_markers)
print(len(trg_toks_after_markers))

In [ ]:
'''
TODO: right now, half the word for the insertion order is done when to_insert is filled out, and the other half is done when the markers are
being inserted (with reverse). Since there there's already funky stuff going on in the order of the markers in to_insert (for disambiguation 
for the same insertion idx), it would make more sense to just do all the ordering when to_insert is being filled out, i.e. the order of a list
in to_insert is the order they need to be inserted in
on the other hand, the current way might be more human-readable
'''

'''Reinsert markers'''
to_insert = [[] for _ in vrefs]

# Collect the markers to be inserted
for i, (mark, next_trg_tok) in enumerate(zip(inline_markers, trg_toks_after_markers)):
    sent_idx, _, marker = mark
    if next_trg_tok >= len(trg_word_tok_ranges[sent_idx]): # TODO: this shouldn't happen
        insert_idx = len(trg_sents[sent_idx])
    else:
        insert_idx = trg_word_tok_ranges[sent_idx][next_trg_tok].start

    # figure out the order of the markers in the sentence to handle ambiguity for directly adjacent markers
    insert_place = 0
    while insert_place < len(to_insert[sent_idx]) and to_insert[sent_idx][insert_place][0] <= insert_idx:
        insert_place += 1

    to_insert[sent_idx].insert(insert_place, (insert_idx, marker))

'''create rows for each paragraph marker and insert character markers back into text'''
# Construct rows to update the USFM file with
rows = []
for sent_idx, (ref, trg_sent) in enumerate(zip(vrefs, trg_sents)):
    row_texts = []
    attach_to_prev = False # TODO: better name
    prev_is_end = False # hacky
    for insert_idx, marker in reversed(to_insert[sent_idx]):
        is_char_marker = src_settings.stylesheet.get_tag(marker.strip(" \\+*")).style_type == UsfmStyleType.CHARACTER
        row_text = (marker if is_char_marker else "") \
                    + (" " if "*" in marker and insert_idx < len(trg_sent) and trg_sent[insert_idx].isalpha() else "") \
                    + trg_sent[insert_idx:]

        if attach_to_prev:
            # don't want a space before end marker
            if prev_is_end and len(row_text) > 0 and row_text[-1] == " ": # hacky
                row_text = row_text[:-1]
            # append text segments instead of creating a new one since previous segment (really the next segment bc iterating backwards) is not its own paragraph
            row_texts[0] = row_text + row_texts[0]
        else:
            row_texts.insert(0, row_text)

        # only paragraph markers get their own rows, so all segments split by character markers and their end markers need to be rejoined
        attach_to_prev = is_char_marker
        prev_is_end = "*" in marker
        trg_sent = trg_sent[:insert_idx]

    # do the same as above with the text at the beginning of the verse
    if attach_to_prev:
        if prev_is_end and len(trg_sent) > 0 and trg_sent[-1] == " ": # hacky
            trg_sent = trg_sent[:-1]
        row_texts[0] = trg_sent + row_texts[0]
    else:
        rows.append(([ref], trg_sent))

    for row_text in row_texts:
        rows.append(([ref], row_text))

# add footnotes to ends of versess
for i, (ref, sent) in enumerate(rows):
    ref = ref[0]
    if i < len(rows) - 1 and rows[i+1][0][0].verse_ref == ref.verse_ref:
        continue
    while len(ignored_segments) > 0 and ignored_segments[0][0].verse_ref == ref.verse_ref:
        rows[i] = ([ref], rows[i][1] + ignored_segments[0][1])
        ignored_segments.pop(0)

print(len(rows))
# for ref, sent in rows:
#     print(ref, sent)

In [241]:
from machine.corpora import UpdateUsfmParserHandler, parse_usfm, UsfmParserState, UpdateUsfmBehavior

class ParagraphUpdateUsfmParserHandler(UpdateUsfmParserHandler):
    def _collect_tokens(self, state: UsfmParserState) -> None:
        self._tokens.extend(self._new_tokens)
        self._new_tokens.clear()
        while self._token_index <= state.index + state.special_token_count:
            if state.tokens[self._token_index].type == UsfmTokenType.PARAGRAPH and state.tokens[self._token_index].marker != "rem":
                num_text = 0
                rem_offset = 0
                for i in range(len(self._tokens) - 1, -1, -1):
                    if self._tokens[i].type == UsfmTokenType.TEXT:
                        num_text += 1
                    elif self._tokens[i].type == UsfmTokenType.PARAGRAPH and self._tokens[i].marker == "rem":
                        rem_offset += num_text + 1
                        num_text = 0
                    else:
                        break
                if num_text >= 2:
                    self._tokens.insert(-(rem_offset + num_text - 1), state.tokens[self._token_index])
                    self._token_index += 1
                    break # should this be continue instead? is there just no difference bc only 1 paragraph marker is added at a time?
            self._tokens.append(state.tokens[self._token_index])
            self._token_index += 1

'''Update USFM and write out'''
# preserve_whitespace=True doesn't change anything with markers on newlines but it does take care of the \vp\*vp somehow
with open(src_fpath, encoding=src_settings.encoding) as f:
    usfm = f.read()
handler = ParagraphUpdateUsfmParserHandler(rows, behavior=UpdateUsfmBehavior.PREFER_NEW)
parse_usfm(usfm, handler, src_settings.stylesheet, src_settings.versification, preserve_whitespace=False)
usfm_out = handler.get_usfm(src_settings.stylesheet)

with out_fpath.open("w", encoding=src_settings.encoding) as f:
    f.write(usfm_out)